In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from scipy.sparse import csr_matrix, coo_matrix

In [2]:
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [3]:
data = pd.read_csv('retail_train.csv')
data.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [4]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

Создадим датафрейм с покупками юзеров на тестовом датасете (последние 3 недели)

In [5]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


### Random recommendation:

In [6]:
def random_recommendation(items, n=5):
    """Случайные рекомендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [7]:
items = data_train.item_id.unique()
result['random_recommendation'] = result['user_id'].map(lambda x: random_recommendation(items, n=5))
result.head(2)

,user_id,actual,random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[890533, 12301826, 997487, 13944804, 955215]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[947761, 819255, 9530360, 919393, 15863846]"


### Popularity-based recommendation:

In [8]:
def popularity_recommendation(data, n=5):
    """Топ-n популярных товаров"""
    
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    
    recs = popular.head(n).item_id
    
    return recs.tolist()

In [9]:
popular_recs = popularity_recommendation(data_train, n=5)
result['popular_recommendation'] = result['user_id'].map(lambda x: popular_recs)

### Item-Item Recommender / ItemKNN:

In [13]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

In [14]:
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [15]:
# Заведем фиктивный item_id (если юзер покупал товары не из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Для работы с матрицами заведем словари перевода идентификаторов в индексы матриц и обратно

In [16]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [17]:
model = ItemItemRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей
model.fit(sparse_user_item,  # На вход item-user matrix
          show_progress=True)
recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=sparse_user_item,   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=[itemid_to_id[999999]], 
                        recalculate_user=False)

  0%|          | 0/5001 [00:00<?, ?it/s]

In [18]:
result['itemitem'] = result['user_id'].\
    map(lambda x: [id_to_itemid[rec] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[999999]], 
                                    recalculate_user=True)[0]])

### Косинусное сходство и CosineRecommender:

In [19]:
model = CosineRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей
model.fit(csr_matrix(user_item_matrix).tocsr(), 
          show_progress=True)
recs = model.recommend(userid=userid_to_id[1], 
                        user_items=sparse_user_item,   # на вход user-item matrix
                        N=5, 
                        filter_already_liked_items=False, 
                        filter_items=[itemid_to_id[999999]], 
                        recalculate_user=False)

C:\Users\rfk03\AppData\Local\Programs\Python\Python310\lib\site-packages\implicit\utils.py:138: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0049898624420166016 seconds
  warnings.warn(


  0%|          | 0/5001 [00:00<?, ?it/s]

In [20]:
result['cosine'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[999999]], 
                                    recalculate_user=False)[0]])

### TF-IDF взвешивание и TFIDFRecommender:

In [21]:
model = TFIDFRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей
model.fit(csr_matrix(user_item_matrix).tocsr(), 
          show_progress=True)
recs = model.recommend(userid=userid_to_id[1], 
                        user_items=sparse_user_item,   # на вход user-item matrix
                        N=5, 
                        filter_already_liked_items=False, 
                        filter_items=[itemid_to_id[999999]], 
                        recalculate_user=False)

C:\Users\rfk03\AppData\Local\Programs\Python\Python310\lib\site-packages\implicit\utils.py:138: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.004987955093383789 seconds
  warnings.warn(


  0%|          | 0/5001 [00:00<?, ?it/s]

In [22]:
result['tfidf'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[999999]], 
                                    recalculate_user=False)[0]])

In [23]:
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[890533, 12301826, 997487, 13944804, 955215]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1098066, 995242, 981760, 1127831, 1082185]","[961554, 1098066, 1127831, 981760, 1082185]","[961554, 1098066, 1127831, 981760, 1082185]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[947761, 819255, 9530360, 919393, 15863846]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1098066, 995242, 981760, 1127831, 1082185]","[961554, 1098066, 1127831, 981760, 1082185]","[961554, 1098066, 1127831, 981760, 1082185]"


### Измерим качество:

In [24]:
def precision_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    flags = np.isin(recommended_list, bought_list)
    precision = flags.sum() / len(recommended_list) 
    return precision

def recall_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    flags = np.isin(bought_list, recommended_list[:k])
    recall = flags.sum() / len(bought_list)
    return recall

In [25]:
dict_rez = {'random_recommendation':[],'popular_recommendation':[],'itemitem':[],'cosine':[],'tfidf':[]}

In [26]:
dict_rez['random_recommendation'].append(result.apply(lambda row: precision_at_k(row['random_recommendation'], row['actual'],5), axis=1).mean())
dict_rez['random_recommendation'].append(result.apply(lambda row: precision_at_k(row['random_recommendation'], row['actual'],3), axis=1).mean())
dict_rez['random_recommendation'].append(result.apply(lambda row: recall_at_k(row['random_recommendation'], row['actual'],5), axis=1).mean())

dict_rez['popular_recommendation'].append(result.apply(lambda row: precision_at_k(row['popular_recommendation'], row['actual'],5), axis=1).mean())
dict_rez['popular_recommendation'].append(result.apply(lambda row: precision_at_k(row['popular_recommendation'], row['actual'],3), axis=1).mean())
dict_rez['popular_recommendation'].append(result.apply(lambda row: recall_at_k(row['popular_recommendation'], row['actual'],5), axis=1).mean())

dict_rez['itemitem'].append(result.apply(lambda row: precision_at_k(row['itemitem'], row['actual'],5), axis=1).mean())
dict_rez['itemitem'].append(result.apply(lambda row: precision_at_k(row['itemitem'], row['actual'],3), axis=1).mean())
dict_rez['itemitem'].append(result.apply(lambda row: recall_at_k(row['itemitem'], row['actual'],5), axis=1).mean())

dict_rez['cosine'].append(result.apply(lambda row: precision_at_k(row['cosine'], row['actual'],5), axis=1).mean())
dict_rez['cosine'].append(result.apply(lambda row: precision_at_k(row['cosine'], row['actual'],3), axis=1).mean())
dict_rez['cosine'].append(result.apply(lambda row: recall_at_k(row['cosine'], row['actual'],5), axis=1).mean())

dict_rez['tfidf'].append(result.apply(lambda row: precision_at_k(row['tfidf'], row['actual'],5), axis=1).mean())
dict_rez['tfidf'].append(result.apply(lambda row: precision_at_k(row['tfidf'], row['actual'],3), axis=1).mean())
dict_rez['tfidf'].append(result.apply(lambda row: recall_at_k(row['tfidf'], row['actual'],5), axis=1).mean())

In [27]:
df_rez = pd.DataFrame(dict_rez, index=['Precision@5', 'Precision@3', 'Recall@5'])
df_rez

,random_recommendation,popular_recommendation,itemitem,cosine,tfidf
Precision@5,0.000196,0.155240,0.145739,0.135455,0.135455
Precision@3,0.000163,0.137773,0.119654,0.058929,0.058929
Recall@5,0.000013,0.024996,0.016218,0.014031,0.014031


### Вывод: наилучшие результаты по всем трем метрикам показал popular_recommendation.